In [ ]:
import kerrgeopy as kg

In [ ]:
def is_stable_orbit(a, E, L, Q):
    """
    用 kerrgeopy 测试能量 E 是否能形成稳定轨道。
    若报错则返回 False，表示不稳定或无法构造稳定轨道。
    """
    try:
        orbit = kg.StableOrbit.from_constants(a, E, L, Q)
        # 如需做更多检查（比如 orbit.plot(...)）可自行添加
        return True
    except:
        return False


def coarse_search(a, L, Q, e_lower, e_upper, n_points=1000):
    """
    在区间 [e_lower, e_upper] 粗略搜索，获取一个大致的 E_min_coarse 和 E_max_coarse。

    返回:
    --------
    (E_min_coarse, E_max_coarse)
        分别为在此区间内第一个出现的稳定能量(从小到大找) 和
        最后一个出现的稳定能量(从大到小找)。
    若没找到任何稳定轨道，则返回 (None, None)。
    """
    E_values = [
        e_lower + i * (e_upper - e_lower) / (n_points - 1)
        for i in range(n_points)
    ]

    E_min_temp = None
    # 找到第一个稳定轨道 E_min_coarse
    for E in E_values:
        if is_stable_orbit(a, E, L, Q):
            E_min_temp = E
            break

    E_max_temp = None
    # 找到最后一个稳定轨道 E_max_coarse
    for E in reversed(E_values):
        if is_stable_orbit(a, E, L, Q):
            E_max_temp = E
            break

    return (E_min_temp, E_max_temp)


def refine_lower_bound(a, L, Q, start_lower, max_precision=15, steps_per_layer=1000, start_layer=6):
    """
    模拟手工搜下界:
    从 start_lower 开始，逐层减小 step = 10^(-k)，往"更小"方向尝试:
      1) candidate = e_min_current - step
      2) 若 still stable，则更新 e_min_current，否则退回并跳到下一层

    参数:
    -----------
      a, L, Q : Kerr参数
      start_lower: 初始猜测的 E_min （如粗搜结果）
      max_precision: 目标精度(小数点后多少位)
      steps_per_layer: 每层尝试多少次
      start_layer: 从多少位小数开始精细化 (如 6 表示从 10^-6 开始)

    返回:
    -----------
      e_min_current: 逼近到小数点后 max_precision 位的下界
    """

    e_min_current = start_lower

    # 一层层缩小步长
    # 这里的 range(start_layer, max_precision+1) 表示第 start_layer 位到第 max_precision 位
    for k in range(start_layer, max_precision + 1):
        step = 10**(-k)
        print(f"\n[LowerBound] 进入第{k}层精度 (step = {step}), 当前 E_min = {e_min_current:.20f}")

        for i in range(1, steps_per_layer + 1):
            candidate = e_min_current - step
            if is_stable_orbit(a, candidate, L, Q):
                e_min_current = candidate  # 能继续往下
            else:
                # 不稳定了，退回上一个 stable 值，跳出本层
                print(f"  第{i}步, candidate = {candidate:.20f} 不稳定，退回到 {e_min_current:.20f}")
                break

        print(f"  第{k}层结束, E_min 已更新到 {e_min_current:.20f}")

    return e_min_current


def refine_upper_bound(a, L, Q, start_upper, max_precision=15, steps_per_layer=1000, start_layer=6):
    """
    模拟手工搜上界:
    从 start_upper 开始，逐层减小 step = 10^(-k)，往"更大"方向尝试:
      1) candidate = e_max_current + step
      2) 若 still stable，则更新 e_max_current，否则退回并跳到下一层

    参数:
    -----------
      a, L, Q : Kerr参数
      start_upper: 初始猜测的 E_max （如粗搜结果）
      max_precision: 目标精度(小数点后多少位)
      steps_per_layer: 每层尝试多少次
      start_layer: 从多少位小数开始精细化 (如 6 表示从 10^-6 开始)

    返回:
    -----------
      e_max_current: 逼近到小数点后 max_precision 位的上界
    """

    e_max_current = start_upper

    # 一层层缩小步长
    for k in range(start_layer, max_precision + 1):
        step = 10**(-k)
        print(f"\n[UpperBound] 进入第{k}层精度 (step = {step}), 当前 E_max = {e_max_current:.20f}")

        for i in range(1, steps_per_layer + 1):
            candidate = e_max_current + step
            if is_stable_orbit(a, candidate, L, Q):
                e_max_current = candidate  # 能继续往上
            else:
                # 不稳定了，退回上一个 stable 值，跳出本层
                print(f"  第{i}步, candidate = {candidate:.20f} 不稳定，退回到 {e_max_current:.10f}")
                break

        print(f"  第{k}层结束, E_max 已更新到 {e_max_current:.20f}")

    return e_max_current


In [ ]:
if __name__ == "__main__":
    # 以 a=0.995, L=2, Q=0 作为示例
    a = 0.995
    L = 1.8
    Q = 0

    # 第一步：粗搜
    # 你可以自行调整初始区间和点数
    coarse_lower = 0.6
    coarse_upper = 1
    n_points = 1000

    E_min_coarse, E_max_coarse = coarse_search(a, L, Q, coarse_lower, coarse_upper, n_points)
    print("\n===== 粗搜结果 =====")
    print(f"E_min_coarse = {E_min_coarse}")
    print(f"E_max_coarse = {E_max_coarse}")

    if E_min_coarse is None or E_max_coarse is None:
        print("粗搜区间内没有找到任何稳定轨道，可能需要调整范围或增大 n_points。")
    else:
        # 第二步：精细搜下界 (若需要的话)
        # 例如目标精度小数点后 15 位
        # 从 10^-6 层级开始精度逼近，每层最多尝试 1000 步
        refined_min = refine_lower_bound(
            a=a, L=L, Q=Q,
            start_lower=E_min_coarse,
            max_precision=17,
            steps_per_layer=1000,
            start_layer=6
        )

        # 第三步：精细搜上界
        refined_max = refine_upper_bound(
            a=a, L=L, Q=Q,
            start_upper=E_max_coarse,
            max_precision=17,
            steps_per_layer=1000,
            start_layer=6
        )

        # 最后打印
        print("\n===== 最终精细结果 =====")
        print(f"E_min ~ {refined_min:.20f}")
        print(f"E_max ~ {refined_max:.20f}")